# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='../chromedriver_win32/chromedriver.exe')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(1)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print("到達頁面底端")
        break

    last_height = new_height
    print("網頁更新中...")
    
    #time.sleep(0.6)
    #browser.execute_script("window.scrollTo(0, 50000);")   


網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 合庫銀海外拓點在下一城　金管會核准申設休士頓分行

▲金管會今天同意合庫銀向美國申請設立休士頓分行，而第一銀行先前也提出申請，目前向當地主管機關申設中。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
金管會今（24）日同意合作金庫商業銀行向美國申請設立休士頓分行，銀行局表示，目前國銀在美國分支機構共有23家分行、3家子行及2家代表人辦事處。
金管會銀行局指出，合庫銀目前已設有西雅圖、洛杉磯、馬尼拉、香港、蘇州、雪梨、金邊、天津、福州、紐約、長沙、墨爾本等12家分行，比利時台灣聯合銀行1家子行及北京、仰光2家代表人辦事處。
銀行局表示，合庫銀為布建更完整美國營業據點，就近服務當地台商客群，因此規劃於德州設立休士頓分行，以強化海外業務觸角及服務品質，擴大海外營運基礎，持續為該行建構全球金融服務網絡。
根據銀行局統計，目前本國銀行在美國分支機構共有23家分行、3家子行及2家代表人辦事處，國銀已有第一商業銀行經過金管會核准向當地主管機關申設德州休士頓分行，目前也向當地主管機關申設中。
----------------------------------------------------------------------

[1] 郵差工作減輕！小額郵包繳稅用「嗶」嘛Ａ通　明年1月3日起可用台灣Pay繳

▲財政部關務署。（圖／記者吳靜君攝）
記者吳靜君／台北報導
財政部關務署今（24）日表示，為了便利民眾繳納進口郵包稅款，明（2020）年1月3日起，與郵局合作推出「行動支付掃碼繳稅」的服務，民眾只要1分鐘，就可以用行動裝置繳完小額郵包進口稅款。
關務署表示，民眾收到小額郵包進口稅款繳納時，過去都是由郵差幫忙代收，但是為未來民眾就可以使用台灣Pay或者郵保鑣App掃描繳納證上的行動條碼（QR-Code），免輸入銷帳編號、繳款金額等稅費資料，就可以選擇用金融卡支付，民眾只要輸入卡片密碼、圖形驗證碼並且確認繳款資訊後，就可以完成繳款。
關務署解釋，小額郵包完稅價格在新台幣2000元以下不用繳稅費、但是超過5000美元（約新台幣15萬2100元）就要報關和需要報關單，所以小額郵包完稅價格在2000元到15多萬元之間，都可以用台灣Pa

In [6]:
browser.quit()